## Сбор данных

Данные – информация о популярных каналах стримингового серсива Twitch.tv
Для сбора данных необходимо два источника информации:
1. [Socialblade](https://socialblade.com/twitch/top/500/followers)– сайт, предоставляющий списки наиболее популярных стримеров (по количеству подписчиков и по количеству просмотров канала) 
2. [Twitch](https://www.twitch.tv) (данные собираются посредством [API](https://dev.twitch.tv))

In [80]:
# загрузка библиотек
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd

Для начала, сбор информации с Socialblade (названия каналов):

In [84]:
# сначала со вкладки, на которой представлены каналы с наибольшим числом подписчиков:
sbl = requests.get("https://socialblade.com/twitch/top/500/followers")
soup = bs(sbl.text, 'html.parser')

num = np.arange(2, 2500, 5) # данные с таблицы идут вперемешку; такая последовательность поможет собрать названия каналов
d = []

for i in range(500):
    d.append({'name': soup.find_all(class_='table-cell')[num[i]].get_text()})
    df1 = pd.DataFrame(d)


# то же самое для каналов, отсортированных по количеству просмотров:
sbl2 = requests.get("https://socialblade.com/twitch/top/500/channelviews")
soup2 = bs(sbl2.text, 'html.parser')

d2 = []

for i in range(500):
    d2.append({'name': soup2.find_all(class_='table-cell')[num[i]].get_text()})
    df2 = pd.DataFrame(d2)
    
# Лично мне удобнее полученные списки привести в табличный формат (pd) для дальнейших объединения и фильтрации 
frames = [df1, df2]
df = pd.concat(frames)

df=df.reset_index()
df.drop(['index'], axis = 1, inplace = True)
df=df.drop_duplicates() # один канал тможет находиться в обоих списках
df=df.reset_index()
df.drop(['index'], axis = 1, inplace = True)

Теперь можно перейти ко сбору данных с самого Twitch.tv

In [86]:
twitch='https://api.twitch.tv/kraken/channels/'
headers = {"Client-ID": "fq4s3rmzc9erhbws2n3z9u20kqf1rh"}
urls=[]
urls.append(twitch + df.name)

twt = requests.get(urls[0][0],headers=headers)
twt.json()["_links"] # базовый список url, по которым можно (или нельзя) собрать данные

{'self': 'https://api.twitch.tv/kraken/channels/ninja',
 'follows': 'https://api.twitch.tv/kraken/channels/ninja/follows',
 'commercial': 'https://api.twitch.tv/kraken/channels/ninja/commercial',
 'stream_key': 'https://api.twitch.tv/kraken/channels/ninja/stream_key',
 'chat': 'https://api.twitch.tv/kraken/chat/ninja',
 'features': 'https://api.twitch.tv/kraken/channels/ninja/features',
 'subscriptions': 'https://api.twitch.tv/kraken/channels/ninja/subscriptions',
 'editors': 'https://api.twitch.tv/kraken/channels/ninja/editors',
 'teams': 'https://api.twitch.tv/kraken/channels/ninja/teams',
 'videos': 'https://api.twitch.tv/kraken/channels/ninja/videos'}

Как видно выше, API позволяет собрать данные о канале по многим разделам (информация о чате/видео/командах и тд), однако, для доступа к некоторым данным не хватает прав (вероятно, часть данных можно собрать только владельцу того или иного канала).
Тем не менее, наиболее важные ссылки всё ещё позволяют найти необходимые данные:

In [92]:
s=[]
for k in range(735): # После удаления дублирующихся данных осталось 735 каналов (данные могут меняться; собирались 24го октября)
    v = str('https://api.twitch.tv/kraken/channels/' + df.name[k] + '/videos?broadcasts=true') # прошлые прямые трансляции
    twt1 = requests.get(v, headers=headers)
    n = str('https://api.twitch.tv/kraken/channels/' + df.name[k]) # общие сведения о канале
    twt2 = requests.get(n,headers=headers)
    t = str('https://api.twitch.tv/kraken/channels/' + df.name[k] + '/teams') # сообщества или т.н. "команды"
    twt3 = requests.get(t,headers=headers)
    h = str('https://api.twitch.tv/kraken/channels/' + df.name[k] + '/videos') # "яркие" моменты канала
    twt4 = requests.get(h,headers=headers)
    f = str('https://api.twitch.tv/kraken/users/' + df.name[k] + '/follows/channels') # кого фолловит сам владелец канала
    twt5 = requests.get(f,headers=headers)
    if '_total' in twt1.json().keys() and 'broadcaster_language' in twt2.json():
            s.append({'name': df.name[k],  # название канала
                      'views': twt2.json()['views'], # количество просмотров
                      'followers': twt2.json()['followers'], # количество подписчиков
                      'following': twt5.json()['_total'], # количество каналов, на которые подписан пользователь
                      'broadcasts_num': twt1.json()['_total'], # сколько записей с предыдуцщих трансляций сохранено
                      'highlights_num': twt4.json()['_total'], # сколько "ярких моментов" вырезано из записей
                      'language': twt2.json()['broadcaster_language'], # основной язык канала
                      'in_partnership': twt2.json()['partner'], # есть ли партнёрская программа
                      'teams': len(twt3.json()['teams']), # в скольких командах состоит
                      'mature': twt2.json()['mature'], # контент (18+/family-friendly)
                      'creation_year': twt2.json()['created_at'][0:4]}) # год создания канала
    else:
        pass


In [93]:
# Создание датафрейма
df=pd.DataFrame(s)
df=df[df.broadcasts_num != 0] # Socialblade предоставляет данные о каналах так, что один и тот же может появиться несколько раз, 
# если ранее он был переименован. В переименованных версиях, как правило, отсутствуют данные о предыдущих трансляциях
df=df.reset_index()
df.drop(['index'], axis = 1, inplace = True)

In [149]:
# Объединение датасетов

typen=pd.read_csv('type.csv') # переменная, которая собиралась вручную: 
# тип канала (любительская трансляция/полу- профессиональная)

result = pd.merge(df, typen, how='inner', on=['name'])
result.head()

,broadcasts_num,creation_year,followers,following,highlights_num,in_partnership,language,mature,name,teams,views,type
0,455,2011,11932442,247,247,True,en,False,ninja,1,356868818,person
1,871,2012,4672585,70,35,True,en,False,shroud,0,226014173,person
2,60,2015,4439377,168,106,True,en,True,tsm_myth,2,62076447,person
3,66,2011,3320792,185,268,True,en,False,summit1g,2,245213917,channel
4,100,2014,3266302,418,152,True,en,False,tfue,1,48294410,person


In [150]:
result.to_csv('twitchdata.csv')